<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Model Read

In [18]:
#===============================================
# Basic Imports
BASE_FOLDER = '../../'
%run -i ..\..\utility\feature_extractor\JupyterLoad_feature_extractor.py
%run -i ..\..\utility\modeling\JupyterLoad_modeling.py

from sklearn.metrics import roc_auc_score
from tqdm import tqdm
import os
import glob
import seaborn as sns
sns.set()

load feature_extractor_mother
load feature_extractor_mel_spectra
load feature_extractor_psd
load feature_extractor_ICA2
load feature_extractore_pre_nnFilterDenoise
load extractor_diagram_mother
load Simple_FIR_HP
load TimeSliceAppendActivation
load load_data
Load split_data
Load anomaly_detection_models
Load pseudo_supervised_models
Load tensorflow models
Load detection_pipe


In [27]:
ensemble_pipes = {}
paths = {}
for SNR in ['6dB', 'min6dB']:
    for machine in ['fan', 'pump', 'slider', 'valve']:
        for ID in ['00', '02']:
            paths[SNR+machine+ID] = glob.glob(BASE_FOLDER + 'modeling/ensemble/pipes2/*_{}_{}_ID{}*.pkl'.format(SNR,machine,ID))
            ensemble_pipes[SNR+machine+ID] = []
            for path in paths[SNR+machine+ID]:
                with open(path, 'rb') as f:
                    ensemble_pipes[SNR+machine+ID].append(pickle.load(f))

C:\Users\bendi\.conda\envs\mimii-tf2-gpu\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [28]:
ensemble_dfs = {}
for ensemble_key, ensemble in ensemble_pipes.items():
    for i, pipe in enumerate(ensemble):

        if i==0:
            ensemble_dfs[ensemble_key] = pd.DataFrame(pipe.df_test.groupby(by='path').sum().pred_scores.rename(pipe.model.name+'_'+pipe.task['feat_col']+'_pred_scores').reindex())
            ground_truth = pipe.df_test.groupby(by='path').median().abnormal.rename('ground_truth')
            score_unweighted = pd.Series(np.zeros(ensemble_dfs[ensemble_key].iloc[:,0].shape), index=ensemble_dfs[ensemble_key].index)
            score_weighted = score_unweighted
        else:
            ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(pipe.df_test.groupby(by='path').sum().pred_scores.rename(pipe.model.name+'_'+pipe.task['feat_col']+'_pred_scores'))
        
        if pipe.model.name == 'IsoFor':
            if pipe.task['feat_col'] == 'PSD_raw':
                weight = 1.
            else:
                weight = .75
        elif pipe.model.name == 'AutoEnc':
            weight = 1.25
        elif pipe.model.name == 'SVM':
            weight = 1.
        score_weighted = score_weighted + weight * (ensemble_dfs[ensemble_key].iloc[:,-1] - ensemble_dfs[ensemble_key].iloc[:,-1].mean())/ensemble_dfs[ensemble_key].iloc[:,-1].std()
        score_unweighted = score_unweighted + (ensemble_dfs[ensemble_key].iloc[:,-1] - ensemble_dfs[ensemble_key].iloc[:,-1].mean())/ensemble_dfs[ensemble_key].iloc[:,-1].std()
    
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(score_weighted.rename('score_agg_weighted'))
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(score_unweighted.rename('score_agg_unweighted'))
    ensemble_dfs[ensemble_key] = ensemble_dfs[ensemble_key].join(ground_truth.replace(to_replace=1, value=-1).replace(to_replace=0, value=1))

In [30]:
ensemble_dfs[ensemble_key]

,IsoFor_MEL_den_pred_scores,AutoEnc_MEL_den_pred_scores,IsoFor_PSD_raw_pred_scores,SVM_PSD_raw_pred_scores,score_agg_weighted,score_agg_unweighted,ground_truth
path,,,,,,,
\dataset\6dB\fan\id_00\abnormal\00000000.wav,-6.802272,-487.554746,-0.079866,0.009014,-4.090335,-4.188305,-1.0
\dataset\6dB\fan\id_00\abnormal\00000001.wav,-5.690387,-627.194746,-0.032358,0.004656,-4.560671,-4.390258,-1.0
\dataset\6dB\fan\id_00\abnormal\00000002.wav,-9.061854,-518.943522,-0.057166,0.002574,-4.339652,-4.465901,-1.0
\dataset\6dB\fan\id_00\abnormal\00000003.wav,-6.605109,-423.003839,-0.084193,0.029282,-3.504717,-3.701643,-1.0
\dataset\6dB\fan\id_00\abnormal\00000004.wav,-7.235790,-447.375194,-0.048582,0.004249,-3.447683,-3.626888,-1.0
...,...,...,...,...,...,...,...
\dataset\6dB\fan\id_00\normal\00000997.wav,6.789946,-251.296146,0.062577,0.062050,1.245097,1.239386,1.0
\dataset\6dB\fan\id_00\normal\00000999.wav,12.395625,-180.217720,0.145569,NaN,NaN,NaN,1.0
\dataset\6dB\fan\id_00\normal\00001002.wav,7.130915,-290.245030,0.139618,0.911818,5.570515,5.640751,1.0


In [29]:
results = pd.DataFrame([])
for ensemble_key, ensemble in ensemble_dfs.items():
    data = [roc_auc_score(ensemble_dfs[ensemble_key].ground_truth, ensemble_dfs[ensemble_key].score_agg_weighted), 
            roc_auc_score(ensemble_dfs[ensemble_key].ground_truth, ensemble_dfs[ensemble_key].score_agg_unweighted)]

    cols = ['score_aggr_weighted', 'score_aggr_unweighted']
    for pipe in ensemble_pipes[ensemble_key]:
        data.append(pipe.roc_auc)
        cols.append('score_'+pipe.model.name+pipe.task['feat_col'])
    results = results.append(pd.DataFrame(data=[data], columns=cols, index=[ensemble_key]))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
results

In [ ]:
results.score_aggr_weighted.mean()

In [ ]:
results.score_aggr_unweighted.mean()

In [ ]:
results.score_AutoEncMEL_den.mean()

In [ ]:
results.score_IsoForMEL_den.mean()

In [ ]:
results.score_IsoForPSD_raw.mean()

In [ ]:
for col in predictions['6dBpump00'].columns:
    s_tmp = s_tmp + (predictions['min6dBpump00'][col] - predictions['min6dBpump00'][col].mean())/predictions['min6dBpump00'][col].std()
predictions['min6dBpump00'].join(s_tmp.rename('score_agg'))

In [ ]:
s_tmp